In [1]:
#Loading packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow
import keras
from keras.models import load_model
import os
import sys 

#Importign local modules
cwd = os.getcwd()
parent_directory = os.path.abspath(os.path.join(cwd, '..', '..'))
sys.path.append(parent_directory)

from utils.move_encoding import encode_move, decode_move
from utils.board_encoding import encode_board, fen_to_board


In [2]:
#OLD Loading the data
data_file = "cleaned_data_1M.pkl"
data_path = os.path.join('..', '..', 'data/cleaned_data', data_file)
df_full = pd.read_pickle(data_path)
df = df_full[['board', 'encoded_board', 'move', 'encoded_move']]


In [3]:
#Loading the data
folder_name = "10M"
folder_path = f"../../data/cleaned_data/{folder_name}" 

def list_files_in_folder(folder_path):
    
    # Get a list of all files in the folder
    files = os.listdir(folder_path)    
    files_with_path = [os.path.join(folder_path, file) for file in files]
    
    return files_with_path


output_files = list_files_in_folder(folder_path)
df = pd.concat([pd.read_pickle(file) for file in output_files], ignore_index=True)



In [3]:
#Obtaining basic descriptives
obs_count = df.shape[0]
encoded_board_shape = df['encoded_board'][0].shape

encoded_board_max = np.max(df['encoded_board'].apply(lambda x: np.max(x)))
encoded_board_min = np.min(df['encoded_board'].apply(lambda x: np.min(x)))

if encoded_board_min != 0 or encoded_board_max != 1:
    print('Check the encoded board values')
else:
    pass
    
print(f'There are {obs_count} observations')

There are 947959 observations


In [4]:
#Converting data to input for the model
x = df['encoded_board']
y = df['encoded_move']
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

x_train = x_train.to_numpy()
x_val = x_val.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

for i in range(len(x_train)):
    x_train[i] = x_train[i].reshape(896)
    
x_train = np.array(x_train.tolist()).astype('float32')

for i in range(len(x_val)):
    x_val[i] = x_val[i].reshape(896)
    
x_val = np.array(x_val.tolist()).astype('float32')

y_train = y_train.astype('float32')
y_val = y_val.astype('float32')


In [5]:
input_shape = x_train[0].shape[0]
y_max = 4672 #Number of possible moves

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(896,)),
    keras.layers.Dense(units=1500, activation='relu'),
    keras.layers.Dense(units=1000, activation='relu'),
    keras.layers.Dense(units=750, activation='relu'),
    keras.layers.Dense(units=500, activation='relu'),
    keras.layers.Dense(units=y_max, activation='softmax')
])

model.compile(
    optimizer='Adam',
    loss='SparseCategoricalCrossentropy', #Because y is an integer representation
    metrics=['accuracy']
)

model.fit(
    x=x_train,
    y=y_train,
    epochs=20
)

Epoch 1/20
20737/20737 [==============================] - 1672s 81ms/step - loss: 5.9437 - accuracy: 0.0347
Epoch 2/20
20737/20737 [==============================] - 1714s 83ms/step - loss: 5.4849 - accuracy: 0.0640
Epoch 3/20
20737/20737 [==============================] - 1723s 83ms/step - loss: 5.2820 - accuracy: 0.0760
Epoch 4/20
20737/20737 [==============================] - 1722s 83ms/step - loss: 5.1569 - accuracy: 0.0841
Epoch 5/20
20737/20737 [==============================] - 1664s 80ms/step - loss: 5.0653 - accuracy: 0.0898
Epoch 6/20
20737/20737 [==============================] - 1744s 84ms/step - loss: 4.9892 - accuracy: 0.0950
Epoch 7/20
20737/20737 [==============================] - 1751s 84ms/step - loss: 4.9205 - accuracy: 0.0995
Epoch 8/20
20737/20737 [==============================] - 1745s 84ms/step - loss: 4.8650 - accuracy: 0.1035
Epoch 9/20
20737/20737 [==============================] - 1749s 84ms/step - loss: 4.8123 - accuracy: 0.1068
Epoch 10/20
20737/20737 [===

In [6]:
model.save('../saved_models/initial_model.h5')